In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import pytz

from models.gan_models import *
import yfinance as yf
import MetaTrader5 as mt5

In [ ]:
experiment = Experiment(
  api_key="vpNJF6XOWcHS6HqH9ZFEjwRcD",
  project_name="tail-price",
  workspace="artaasd95"
)

In [31]:
mt5.initialize()

print('loading current tf data')
utc_from = datetime(2021, 1, 1, tzinfo=pytz.timezone("Asia/Nicosia"))
utc_to = datetime.now(pytz.timezone("Asia/Nicosia"))

data = mt5.copy_rates_range('XAUUSD', mt5.TIMEFRAME_D1, utc_from, utc_to)
data = pd.DataFrame(data)
time_data = data.time
data.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)

loading current tf data


In [ ]:
data.to_csv('xau_2021_D1.csv')

In [ ]:
data = pd.read_csv('xau_2021_D1.csv')

In [32]:
data

,time,open,high,low,close
0,1609718400,1904.48,1944.33,1900.62,1943.10
1,1609804800,1941.22,1952.90,1934.13,1949.87
2,1609891200,1948.82,1959.30,1900.65,1918.62
3,1609977600,1917.82,1927.54,1906.69,1913.35
4,1610064000,1913.52,1917.38,1828.05,1848.58
...,...,...,...,...,...
932,1723420800,2428.05,2472.17,2423.83,2472.06
933,1723507200,2470.68,2476.97,2458.46,2465.41
934,1723593600,2466.51,2479.94,2438.05,2447.66
935,1723680000,2446.85,2470.08,2432.15,2456.19


In [33]:
input_size = 1
hidden_size = 512
seq_length = 50
num_layers = 8
batch_size = 256
num_epochs = 100
learning_rate = 0.2

# Initialize models
main_gen = MainGenerator(input_size, hidden_size, num_layers, 0.5, 1)
noise_gen = NoiseGenerator(input_size, 1024, 1, 0.3)
discriminator = Discriminator(input_size, hidden_size, 10, 0.3)

In [ ]:
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

In [ ]:
main_gen.to(device)
noise_gen.to(device)
discriminator.to(device)

In [34]:
optimizer_G = torch.optim.AdamW(list(main_gen.parameters()) + list(noise_gen.parameters()), lr=learning_rate)
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=learning_rate)
optimizer_noise = torch.optim.AdamW(noise_gen.parameters(), lr=learning_rate)

optim_g_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, 0.1)
optim_d_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, 0.1)
optim_noise_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_noise, 0.1)

In [35]:
cauchy_dist = torch.distributions.cauchy.Cauchy(loc=0, scale=0.5)

In [36]:
adversarial_loss = torch.nn.KLDivLoss()

In [ ]:
train_data = data[:int(len(data)*0.7)]
train_data = train_data.reset_index(drop=True)
test_data = data[int(len(data)*0.7):]
test_data = test_data.reset_index(drop=True)

In [37]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    for idx in range(len(train_data)):
        price = torch.tensor(train_data.iloc[idx].close, dtype=torch.float).reshape(1,price.shape[0])
        price.to(device)
        # Generate random noise inputs
        z1 = cauchy_dist.sample([1])
        z1.to(device)
        #z2 = cauchy_dist.sample([1])
        
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_D.zero_grad()
        d_loss = adversarial_loss(discriminator(fake_data.detach()), price)
        d_loss.backward()
        optimizer_D.step()
        
        # Train generators
        optimizer_G.zero_grad()
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        d_loss = adversarial_loss(discriminator(fake_data), price)
        g_loss = adversarial_loss(fake_data, price)
        noise_loss = adversarial_loss(fake_noise, z1)
        total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
        total_loss.backward()
        optimizer_G.step()
        optimizer_noise.step()
    
    optim_g_sched.step()
    optim_d_sched.step()
    optim_noise_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
    experiment.log_metric('Main Generator\Train', g_loss)
    experiment.log_metric('Noise Generator\Train', noise_loss)
    experiment.log_metric('Discriminator\Train', d_loss)
    experiment.log_metric('Total\Train', total_loss)
    log_model(experiment, model=main_gen, model_name="Main Generator")
    log_model(experiment, model=noise_gen, model_name="Noise Generator")
    log_model(experiment, model=discriminator, model_name="Discriminator")


  0%|          | 0/150 [00:00<?, ?it/s]c:\Users\arta\anaconda3\envs\torch\Lib\site-packages\torch\nn\functional.py:2976: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
  0%|          | 0/150 [41:18<?, ?it/s]

Epoch [1/150]  Discriminator Loss: 17113.4844  Generator Loss: -89089280.0000


NameError: name 'experiment' is not defined

In [ ]:
# Test loop
for idx in range(len(test_data)):
    price = torch.tensor(test_data.iloc[idx].close, dtype=torch.float).reshape(1,price.shape[0])
    price.to(device)
    # Generate random noise inputs
    z1 = cauchy_dist.sample([1])
    z1.to(device)
    #z2 = cauchy_dist.sample([1])
    main_gen.eval()
    noise_gen.eval()
    discriminator.eval()
    # Generate fake data
    fake_main = main_gen(price)
    fake_noise = noise_gen(z1)
    fake_data = fake_main + fake_noise


    d_loss = adversarial_loss(discriminator(fake_data), price)
    g_loss = adversarial_loss(fake_data, price)
    noise_loss = adversarial_loss(fake_noise, z1)
    total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
    total_loss.backward()
    optimizer_G.step()
    optimizer_noise.step()

optim_g_sched.step()
optim_d_sched.step()
optim_noise_sched.step()

print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
experiment.log_metric('Main Generator\Test', g_loss)
experiment.log_metric('Noise Generator\Test', noise_loss)
experiment.log_metric('Discriminator\Test', d_loss)
experiment.log_metric('Total\Test', total_loss)
# log_model(experiment, model=main_gen, model_name="Main Generator")
# log_model(experiment, model=noise_gen, model_name="Noise Generator")
# log_model(experiment, model=discriminator, model_name="Discriminator")


In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    for idx in range(len(data)):
        price = torch.tensor(data.iloc[idx].close, dtype=torch.float).reshape(1,price.shape[0])
        price.to(device)
        # Generate random noise inputs
        z1 = cauchy_dist.sample([1])
        z1.to(device)
        #z2 = cauchy_dist.sample([1])
        
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_D.zero_grad()
        d_loss = adversarial_loss(discriminator(fake_data.detach()), price)
        d_loss.backward()
        optimizer_D.step()
        
        # Train generators
        optimizer_G.zero_grad()
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        d_loss = adversarial_loss(discriminator(fake_data), price)
        g_loss = adversarial_loss(fake_data, price)
        noise_loss = adversarial_loss(fake_noise, z1)
        total_loss = 0.5 * ((0.8 * g_loss + 0.2 * noise_loss) + d_loss)
        total_loss.backward()
        optimizer_G.step()
        optimizer_noise.step()
    
    optim_g_sched.step()
    optim_d_sched.step()
    optim_noise_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
    experiment.log_metric('Main Generator\WholeTrain', g_loss)
    experiment.log_metric('Noise Generator\WholeTrain', noise_loss)
    experiment.log_metric('Discriminator\WholeTrain', d_loss)
    experiment.log_metric('Total\WholeTrain', total_loss)
    log_model(experiment, model=main_gen, model_name="Main Generator")
    log_model(experiment, model=noise_gen, model_name="Noise Generator")
    log_model(experiment, model=discriminator, model_name="Discriminator")


In [ ]:
torch.save(main_gen, 'checkpoints/main_gen_xau.pth')
torch.save(noise_gen, 'checkpoints/noise_gen_xau.pth')
torch.save(discriminator, 'checkpoints/discriminator_xau.pth')